In [2]:
import rpyc
import logging
import time
import cv2
import numpy as np
import base64
from IPython.display import display, Image  # No need for clear_output here
import ipywidgets as widgets
import os
import csv
import datetime
import torchvision.transforms as transforms
from PIL import Image
import random
import config


# --- Setup Logging ---
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('JetBotClient')

# --- Image Transformation ---
# Transformations *before* saving to disk (for consistency with training)
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


class RemoteJetBot:
    def __init__(self, ip_address, port=18861):
        logger.info(f"Connecting to JetBot at {ip_address}:{port}")
        try:
            self.conn = rpyc.connect(
                ip_address,
                port,
                config={
                    'sync_request_timeout': 30,
                    'allow_all_attrs': True
                }
            )
            logger.info("Connected successfully!")
            # Initialize video window
            self.image_widget = widgets.Image(
                format='jpeg',
                width=400,
                height=300,
            )
            display(self.image_widget)
        except Exception as e:
            logger.error(f"Connection failed: {str(e)}")
            raise

    def get_frame(self):
        """Get a single frame from the camera and display it"""
        try:
            # Get frame from server
            jpg_as_text = self.conn.root.get_camera_frame()
            if jpg_as_text:
                # Decode base64 string directly to bytes
                jpg_bytes = base64.b64decode(jpg_as_text)
                # Update the image widget
                self.image_widget.value = jpg_bytes

                # Convert to NumPy array (for saving)
                npimg = np.frombuffer(jpg_bytes, dtype=np.uint8)
                frame = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
                return frame  # Return the frame as a NumPy array
            return None

        except Exception as e:
            logger.error(f"Error getting frame: {str(e)}")
            return None

    def set_motors(self, left_speed, right_speed):
        try:
            logger.debug(f"Sending motor command: left={left_speed}, right={right_speed}")
            result = self.conn.root.set_motors(float(left_speed), float(right_speed))
            logger.debug("Command sent successfully")
            return result
        except Exception as e:
            logger.error(f"Error sending motor command: {str(e)}")
            raise

    def cleanup(self):
        try:
            logger.debug("Cleaning up connection")
            if hasattr(self, 'conn'):
                self.set_motors(0, 0)  # Stop motors
                self.conn.close()
            logger.info("Cleanup completed")
        except Exception as e:
            logger.error(f"Error during cleanup: {str(e)}")


def generate_random_actions(num_actions, possible_speeds, min_duration, max_duration):
    actions = []
    for _ in range(num_actions):
        speed = random.choice(possible_speeds)
        duration = random.uniform(min_duration, max_duration)  # Use uniform for continuous range
        actions.append((speed, duration))
    return actions

def record_data(jetbot, actions, target_fps, output_dir, image_dir, csv_path):
    """
    Records data, alternating between actions.  Includes debug prints.
    """
    print("Starting data recording...")  # Initial print statement
    with open(csv_path, 'w', newline='') as csvfile:
        fieldnames = ['image_path', 'timestamp', 'action']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        print("CSV header written.")

        target_interval = 1.0 / target_fps
        image_count = 0

        for action, duration in actions:
            print(f"Starting action: {action} for duration: {duration}")
            jetbot.set_motors(action, 0)  # Set motor speed.
            start_time = time.time()

            while time.time() - start_time < duration:
                frame_start_time = time.perf_counter()

                frame = jetbot.get_frame()
                if frame is None:
                    continue

                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image = Image.fromarray(frame)
                image = transform(image)

                timestamp = time.time()
                image_filename = f"image_{image_count:05d}.jpg"
                image_path = os.path.join(image_dir, image_filename)

                pil_image = transforms.ToPILImage()((image + 1) / 2)
                pil_image.save(image_path)

                writer.writerow({'image_path': image_path, 'timestamp': timestamp, 'action': action})
                image_count += 1

                frame_end_time = time.perf_counter()
                elapsed_time = frame_end_time - frame_start_time
                sleep_time = target_interval - elapsed_time

                if sleep_time > 0:
                    time.sleep(sleep_time)

    print(f"Data recording complete. Total images: {image_count}")




In [3]:

# --- Configuration ---
JETBOT_IP = '192.168.68.54'  # Replace with your Jetbot's IP address
IMAGE_SIZE = 224  # Use 224x224 images, don't use constant from config file since there may be resizing, or rename this and put it there
TARGET_FPS = 30
POSSIBLE_SPEEDS = [0.0, 0.1]
MIN_DURATION = 2.0  # Seconds
MAX_DURATION = 5.0  # Seconds
NUM_ACTIONS = 20 #How many total actions to do

In [4]:
jetbot = RemoteJetBot(JETBOT_IP)

try:
    random_actions = generate_random_actions(NUM_ACTIONS, POSSIBLE_SPEEDS, MIN_DURATION, MAX_DURATION)
    print(random_actions)

    # Record data
    record_data(jetbot, random_actions, TARGET_FPS, config.DATA_DIR, config.IMAGE_DIR, config.CSV_PATH)
finally:
    jetbot.cleanup()  # Stop motors and close connection

INFO:JetBotClient:Connecting to JetBot at 192.168.68.54:18861
INFO:JetBotClient:Connected successfully!


Image(value=b'', format='jpeg', height='300', width='400')

DEBUG:JetBotClient:Sending motor command: left=0.0, right=0
DEBUG:JetBotClient:Command sent successfully


[(0.0, 4.034340651685954), (0.0, 3.990489380237825), (0.1, 4.435173537996854), (0.1, 3.4714181567471005), (0.1, 4.643543862823519), (0.1, 4.305233740056094), (0.1, 3.944607375956734), (0.0, 4.499265649163936), (0.1, 3.109759593452303), (0.1, 2.095439108693426), (0.0, 2.3819462813301167), (0.1, 2.9388610541740183), (0.0, 4.081020261138724), (0.1, 2.792746946330512), (0.0, 2.5914472253285323), (0.0, 3.8586250129630777), (0.1, 2.953829215890508), (0.0, 3.8094200403703518), (0.1, 2.3116222746564175), (0.0, 4.122961924677755)]
Starting data recording...
CSV header written.
Starting action: 0.0 for duration: 4.034340651685954


DEBUG:JetBotClient:Sending motor command: left=0.0, right=0
DEBUG:JetBotClient:Command sent successfully


...

Starting action: 0.1 for duration: 2.3116222746564175


DEBUG:JetBotClient:Sending motor command: left=0.0, right=0
DEBUG:JetBotClient:Command sent successfully


Starting action: 0.0 for duration: 4.122961924677755


DEBUG:JetBotClient:Cleaning up connection
DEBUG:JetBotClient:Sending motor command: left=0, right=0
DEBUG:JetBotClient:Command sent successfully
INFO:JetBotClient:Cleanup completed


Data recording complete. Total images: 2084


In [8]:
jetbot = RemoteJetBot(JETBOT_IP)

INFO:JetBotClient:Connecting to JetBot at 192.168.68.54:18861
INFO:JetBotClient:Connected successfully!


Image(value=b'', format='jpeg', height='300', width='400')

In [10]:
jetbot.get_frame()

array([[[195, 163, 210],
        [193, 166, 210],
        [183, 168, 206],
        ...,
        [100,  59,  66],
        [115,  91, 101],
        [ 91,  75,  86]],

       [[192, 160, 207],
        [193, 166, 210],
        [181, 166, 204],
        ...,
        [ 98,  63,  67],
        [108,  87,  96],
        [100,  86,  98]],

       [[187, 158, 203],
        [187, 162, 206],
        [183, 165, 206],
        ...,
        [ 87,  62,  60],
        [101,  86,  94],
        [ 96,  87, 100]],

       ...,

       [[ 91, 101, 119],
        [103, 109, 128],
        [118, 112, 135],
        ...,
        [140, 135, 164],
        [138, 134, 163],
        [140, 139, 165]],

       [[102,  99, 124],
        [107, 103, 128],
        [113, 101, 129],
        ...,
        [146, 137, 164],
        [134, 129, 156],
        [134, 131, 156]],

       [[124, 117, 144],
        [115, 106, 133],
        [116, 100, 131],
        ...,
        [137, 126, 152],
        [131, 127, 152],
        [135, 132, 157]]

In [11]:
jetbot.set_motors(.115,0)

DEBUG:JetBotClient:Sending motor command: left=0.115, right=0
DEBUG:JetBotClient:Command sent successfully


True

In [12]:
jetbot.set_motors(0,0)

DEBUG:JetBotClient:Sending motor command: left=0, right=0
DEBUG:JetBotClient:Command sent successfully


True

In [24]:
jetbot.cleanup()

DEBUG:JetBotClient:Cleaning up connection
DEBUG:JetBotClient:Sending motor command: left=0, right=0
DEBUG:JetBotClient:Command sent successfully
INFO:JetBotClient:Cleanup completed
